In [2]:
import re
import time
from datetime import datetime
from pathlib import Path
from urllib.parse import parse_qs, urlencode, urljoin, urlparse, urlunparse

import pandas as pd
import requests
from bs4 import BeautifulSoup

# ---- Search settings ----
keyword = "data analyst"
location = ""  # optional: e.g. "bangkok"

MAX_PAGES = 10
SLEEP_SECONDS = 1.0

keyword_slug = "-".join(keyword.lower().split())
base_url = f"https://th.jobsdb.com/th/{keyword_slug}-jobs"

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/121.0.0.0 Safari/537.36"
    ),
    "Accept-Language": "en-US,en;q=0.9,th;q=0.8",
}

THAI_PROVINCES = [
    "กรุงเทพมหานคร", "กระบี่", "กาญจนบุรี", "กาฬสินธุ์", "กำแพงเพชร", "ขอนแก่น", "จันทบุรี", "ฉะเชิงเทรา", "ชลบุรี", "ชัยนาท",
    "ชัยภูมิ", "ชุมพร", "เชียงราย", "เชียงใหม่", "ตรัง", "ตราด", "ตาก", "นครนายก", "นครปฐม", "นครพนม", "นครราชสีมา",
    "นครศรีธรรมราช", "นครสวรรค์", "นนทบุรี", "นราธิวาส", "น่าน", "บึงกาฬ", "บุรีรัมย์", "ปทุมธานี", "ประจวบคีรีขันธ์",
    "ปราจีนบุรี", "ปัตตานี", "พระนครศรีอยุธยา", "พังงา", "พัทลุง", "พิจิตร", "พิษณุโลก", "เพชรบุรี", "เพชรบูรณ์", "แพร่",
    "พะเยา", "ภูเก็ต", "มหาสารคาม", "มุกดาหาร", "แม่ฮ่องสอน", "ยะลา", "ยโสธร", "ร้อยเอ็ด", "ระนอง", "ระยอง", "ราชบุรี",
    "ลพบุรี", "ลำปาง", "ลำพูน", "เลย", "ศรีสะเกษ", "สกลนคร", "สงขลา", "สตูล", "สมุทรปราการ", "สมุทรสงคราม", "สมุทรสาคร",
    "สระแก้ว", "สระบุรี", "สิงห์บุรี", "สุโขทัย", "สุพรรณบุรี", "สุราษฎร์ธานี", "สุรินทร์", "หนองคาย", "หนองบัวลำภู", "อ่างทอง",
    "อุดรธานี", "อุตรดิตถ์", "อุทัยธานี", "อุบลราชธานี", "อำนาจเจริญ",
]

EN_TO_THAI_PROVINCE = {
    "bangkok": "กรุงเทพมหานคร",
    "nonthaburi": "นนทบุรี",
    "pathum thani": "ปทุมธานี",
    "samut prakan": "สมุทรปราการ",
    "chon buri": "ชลบุรี",
    "chonburi": "ชลบุรี",
    "rayong": "ระยอง",
    "chiang mai": "เชียงใหม่",
    "chiang rai": "เชียงราย",
    "phuket": "ภูเก็ต",
    "khon kaen": "ขอนแก่น",
    "nakhon ratchasima": "นครราชสีมา",
    "korat": "นครราชสีมา",
    "songkhla": "สงขลา",
    "surat thani": "สุราษฎร์ธานี",
    "udon thani": "อุดรธานี",
    "ubon ratchathani": "อุบลราชธานี",
    "nakhon pathom": "นครปฐม",
    "phra nakhon si ayutthaya": "พระนครศรีอยุธยา",
    "ayutthaya": "พระนครศรีอยุธยา",
}


def update_query_in_url(url: str, **params) -> str:
    parsed = urlparse(url)
    query = parse_qs(parsed.query)
    for key, value in params.items():
        query[key] = [str(value)]
    new_query = urlencode(query, doseq=True)
    return urlunparse((parsed.scheme, parsed.netloc, parsed.path, parsed.params, new_query, parsed.fragment))


def clean_text(text: str) -> str:
    return re.sub(r"\s+", " ", (text or "")).strip()


def normalize_for_match(text: str) -> str:
    return re.sub(r"[^a-z0-9]+", " ", (text or "").lower()).strip()


def keyword_match_groups_from_query(search_keyword: str) -> list[list[str]]:
    term_variants = {
        "data": ["data"],
        "scientist": ["scientist", "science", "scien", "scient"],
        "science": ["science", "scientist", "scien", "scient"],
        "engineer": ["engineer", "engineering", "eng"],
        "analyst": ["analyst", "analytics", "analysis"],
        "developer": ["developer", "development", "dev"],
    }

    tokens = [token for token in normalize_for_match(search_keyword).split() if token]
    groups = []

    for token in tokens:
        groups.append(term_variants.get(token, [token]))

    return groups


def title_matches_keyword(title: str, keyword_groups: list[list[str]]) -> bool:
    if not keyword_groups:
        return True
    title_norm = normalize_for_match(title)
    return all(any(variant in title_norm for variant in group) for group in keyword_groups)


def extract_salary(text: str) -> str:
    patterns = [
        r"THB\s*[\d,]+\s*[-–]\s*THB\s*[\d,]+",
        r"THB\s*[\d,]+",
        r"[\d,]+\s*[-–]\s*[\d,]+\s*บาท",
        r"[\d,]+\s*บาท",
        r"Negotiable|ไม่ระบุเงินเดือน|ตามตกลง|ตามประสบการณ์",
    ]
    for pattern in patterns:
        match = re.search(pattern, text, flags=re.IGNORECASE)
        if match:
            return clean_text(match.group(0))
    return ""


def is_probable_salary(text: str) -> bool:
    if not text:
        return False
    text_norm = text.lower()
    salary_keywords = ["thb", "บาท", "salary", "negotiable", "ตามตกลง", "ตามประสบการณ์"]
    if any(key in text_norm for key in salary_keywords):
        return True
    return bool(re.search(r"\d", text_norm) and re.search(r"[-–]", text_norm))


def guess_province_name(location_text: str) -> str:
    location_clean = clean_text(location_text)
    if not location_clean:
        return ""

    for province in THAI_PROVINCES:
        if province in location_clean:
            return province

    location_lower = location_clean.lower()
    for english_name, thai_name in EN_TO_THAI_PROVINCE.items():
        if re.search(rf"\b{re.escape(english_name)}\b", location_lower):
            return thai_name

    parts = [clean_text(part) for part in re.split(r",|\||/", location_clean) if clean_text(part)]
    if not parts:
        return ""

    tail = parts[-1]
    tail = re.sub(r"^(เขต|อ\.|อำเภอ|จ\.|จังหวัด)\s*", "", tail).strip()
    return tail


def parse_card(card, page_num: int, search_keyword: str) -> dict:
    title_el = card.select_one("a[data-automation='jobTitle']")
    company_el = card.select_one("a[data-automation='jobCompany'], [data-automation='jobCompany']")
    location_el = card.select_one("a[data-automation='jobLocation'], [data-automation='jobCardLocation']")
    date_el = card.select_one("[data-automation='jobListingDate']")
    salary_el = card.select_one("[data-automation='jobSalary']")
    overlay_link_el = card.select_one("a[data-automation='job-list-item-link-overlay'][href]")

    title = clean_text(title_el.get_text(" ", strip=True) if title_el else "")
    company = clean_text(company_el.get_text(" ", strip=True) if company_el else "")
    location_name = clean_text(location_el.get_text(" ", strip=True) if location_el else "")
    posted_date = clean_text(date_el.get_text(" ", strip=True) if date_el else "")

    salary_candidate = clean_text(salary_el.get_text(" ", strip=True) if salary_el else "")
    salary = salary_candidate if is_probable_salary(salary_candidate) else ""

    href = ""
    if overlay_link_el:
        href = overlay_link_el.get("href", "")
    elif title_el and title_el.get("href"):
        href = title_el.get("href", "")
    job_url = urljoin("https://th.jobsdb.com", href) if href else ""

    raw_text = clean_text(card.get_text("\n", strip=True))
    if not salary:
        salary = extract_salary(raw_text)

    province_name = guess_province_name(location_name)

    return {
        "keyword": search_keyword,
        "province_code": "",
        "province_name": province_name,
        "page": page_num,
        "job_title": title,
        "company": company,
        "location": location_name,
        "salary": salary,
        "posted_date": posted_date,
        "job_url": job_url,
        "raw_text": raw_text,
    }


def scrape_jobsdb(search_url: str, search_location: str, max_pages: int = 10, sleep_seconds: float = 1.0) -> pd.DataFrame:
    keyword_groups = keyword_match_groups_from_query(keyword)

    all_rows = []
    seen_urls = set()

    print(f"[Search] Starting JobsDB crawl: max_pages={max_pages}")

    for page_num in range(1, max_pages + 1):
        page_url = update_query_in_url(search_url, page=page_num)
        if search_location.strip():
            page_url = update_query_in_url(page_url, where=search_location.strip())

        print(f"[Search] Page {page_num}/{max_pages} -> request")
        response = requests.get(page_url, headers=headers, timeout=30)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, "html.parser")
        cards = soup.select("article[data-testid='job-card'], article[data-automation='normalJob']")
        print(f"[Search] Page {page_num}/{max_pages} -> found cards: {len(cards)}")

        if not cards:
            print(f"[Search] Page {page_num}/{max_pages} -> no cards, stopping")
            break

        page_rows = []
        for card in cards:
            row = parse_card(card, page_num=page_num, search_keyword=keyword)
            if not row["job_title"] or not row["job_url"]:
                continue
            if not title_matches_keyword(row["job_title"], keyword_groups):
                continue
            if row["job_url"] in seen_urls:
                continue

            seen_urls.add(row["job_url"])
            page_rows.append(row)

        if not page_rows:
            print(f"[Search] Page {page_num}/{max_pages} -> no keyword matches, stopping")
            break

        all_rows.extend(page_rows)
        print(f"[Search] Page {page_num}/{max_pages} -> kept {len(page_rows)} | cumulative={len(all_rows)}")

        if sleep_seconds > 0:
            time.sleep(sleep_seconds)

    df = pd.DataFrame(all_rows)

    if not df.empty:
        df = df[
            [
                "keyword",
                "province_code",
                "province_name",
                "page",
                "job_title",
                "company",
                "location",
                "salary",
                "posted_date",
                "job_url",
                "raw_text",
            ]
        ].drop_duplicates(subset=["job_url"])

    return df


jobs_df = scrape_jobsdb(
    base_url,
    search_location=location,
    max_pages=MAX_PAGES,
    sleep_seconds=SLEEP_SECONDS,
)
print(f"\nTotal unique jobs: {len(jobs_df)}")

output_file = f"jobsdb_{keyword_slug}_{datetime.now().strftime('%Y%m%d')}.csv"
output_path = Path(output_file)
jobs_df.to_csv(output_path, index=False, encoding="utf-8-sig")
print(f"Saved to: {output_path.resolve()}")

jobs_df.head(20)

[Search] Starting JobsDB crawl: max_pages=10
[Search] Page 1/10 -> request
[Search] Page 1/10 -> found cards: 32
[Search] Page 1/10 -> kept 23 | cumulative=23
[Search] Page 2/10 -> request
[Search] Page 2/10 -> found cards: 32
[Search] Page 2/10 -> kept 14 | cumulative=37
[Search] Page 3/10 -> request
[Search] Page 3/10 -> found cards: 32
[Search] Page 3/10 -> kept 12 | cumulative=49
[Search] Page 4/10 -> request
[Search] Page 4/10 -> found cards: 32
[Search] Page 4/10 -> kept 5 | cumulative=54
[Search] Page 5/10 -> request
[Search] Page 5/10 -> found cards: 32
[Search] Page 5/10 -> kept 5 | cumulative=59
[Search] Page 6/10 -> request
[Search] Page 6/10 -> found cards: 32
[Search] Page 6/10 -> kept 2 | cumulative=61
[Search] Page 7/10 -> request
[Search] Page 7/10 -> found cards: 32
[Search] Page 7/10 -> kept 2 | cumulative=63
[Search] Page 8/10 -> request
[Search] Page 8/10 -> found cards: 32
[Search] Page 8/10 -> kept 2 | cumulative=65
[Search] Page 9/10 -> request
[Search] Page 9/10

,keyword,province_code,province_name,page,job_title,company,location,salary,posted_date,job_url,raw_text
0,data analyst,,กรุงเทพมหานคร,1,Data Analyst,"Protech Transfer Co., Ltd.",ยานนาวา กรุงเทพมหานคร,,4 ชั่วโมงที่ผ่านมา,https://th.jobsdb.com/th/job/90386899?type=sta...,Listed more than four days ago Data Analyst ที...
1,data analyst,,กรุงเทพมหานคร,1,Data Claim Analyst (Motor Claim),Krungthai Panich Insurance Public Company Limited,กรุงเทพมหานคร,,3 ชั่วโมงที่ผ่านมา,https://th.jobsdb.com/th/job/90387373?type=sta...,Listed more than three days ago Data Claim Ana...
2,data analyst,,กรุงเทพมหานคร,1,Data Analyst /Operations Report /Customer Care...,Cartrack SEA,บางนา กรุงเทพมหานคร,,1 วันที่ผ่านมา,https://th.jobsdb.com/th/job/89813394?type=sta...,Listed more than one day ago Data Analyst /Ope...
3,data analyst,,กรุงเทพมหานคร,1,Data Analyst,"Focus Media (Thailand) Co., Ltd.",กรุงเทพมหานคร,,6 วันที่ผ่านมา,https://th.jobsdb.com/th/job/90278961?type=sta...,Listed more than six days ago Data Analyst ที่...
4,data analyst,,กรุงเทพมหานคร,1,Data Analyst – Business Decision & Data Quality,Maybank Securities (Thailand) Public Company L...,กรุงเทพมหานคร,,21 วันที่ผ่านมา,https://th.jobsdb.com/th/job/89949693?type=sta...,Listed more than twenty one days ago Data Anal...
5,data analyst,,นนทบุรี,1,Data Analyst (Fraud Management),Triple T Broadband Public Company Limited,ปากเกร็ด นนทบุรี,,3 วันที่ผ่านมา,https://th.jobsdb.com/th/job/90337958?type=sta...,Listed more than three days ago Data Analyst (...
6,data analyst,,กรุงเทพมหานคร,1,Data Analyst,GMM Grammy Public Company Limited,วัฒนา กรุงเทพมหานคร,,5 วันที่ผ่านมา,https://th.jobsdb.com/th/job/90297020?type=sta...,Listed more than five days ago Data Analyst ที...
7,data analyst,,กรุงเทพมหานคร,1,Data Analyst (Food Business),Central Group (Central Pattana Public Company ...,ปทุมวัน กรุงเทพมหานคร,,18 วันที่ผ่านมา,https://th.jobsdb.com/th/job/90012046?type=sta...,Listed more than eighteen days ago Data Analys...
8,data analyst,,กรุงเทพมหานคร,1,Commercial Data Analyst (1-3 yrs.) under Comme...,"Berlin Pharmaceutical Industry Co., Ltd.",กรุงเทพมหานคร,,18 วันที่ผ่านมา,https://th.jobsdb.com/th/job/90031204?type=sta...,Listed more than eighteen days ago Commercial ...
9,data analyst,,กรุงเทพมหานคร,1,เจ้าหน้าที่วิเคราะห์ข้อมูล (MIS/Data Analyst),Muang Thai Life Assurance Public Company Limited,ห้วยขวาง กรุงเทพมหานคร,,11 วันที่ผ่านมา,https://th.jobsdb.com/th/job/90163043?type=sta...,Listed more than eleven days ago เจ้าหน้าที่วิ...


In [ ]:
# Hello world